In [ ]:
#Imports
import csv
import cv2
import numpy as np
from sklearn.cross_validation import train_test_split
import sklearn
import random
from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda,Cropping2D,Dropout,Activation
from keras.layers.convolutional import Convolution2D 
from keras.layers.pooling import MaxPooling2D
from keras.initializations import glorot_normal
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Center0,Left1,Right2,Angle3,Throttle4,Break5,Speed6 (Access from 0-6)
lines=[]
with open("../Data/driving_log.csv") as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        lines.append(line)
#Split into sets
TrainSamples, ValidationSamples=train_test_split(lines,test_size=0.2)
print("DataSet Ready")

In [ ]:
print(len(TrainSamples))
print(len(ValidationSamples))

In [ ]:
#Sample PreProcessing
SamImage=cv2.imread(TrainSamples[0][0])
SamImage=cv2.cvtColor(SamImage,cv2.COLOR_BGR2RGB)
plt.figure(1)
plt.title("Original Image")
plt.imshow(SamImage)
print(SamImage.shape)
SamImage=cv2.resize(SamImage,(160,80),cv2.INTER_CUBIC)
plt.figure(2)
plt.axes().set_aspect('equal')
plt.imshow(SamImage)

In [ ]:
NewShape=[80,160]
def PreProcess(Img):
    Image=Img
    Image=cv2.resize(Image,(NewShape[1],NewShape[0]),cv2.INTER_CUBIC)
    return Image

In [ ]:
#Collect data from files
SteerCorrection=0.25
Multiplier=3
#Generators
def BatchGenerator(Data,BatchSize=1):
    NumSamples=len(Data)
    while 1:
        random.shuffle(Data)                             #Shuffle Data so images come in random order
        for offset in range(0,NumSamples,BatchSize):     #Increment offset by BatchSize
            BatchSample=Data[offset:offset+BatchSize]    #Select batch size number of Data samples
        TrainFeatures=np.zeros((len(BatchSample)*Multiplier,NewShape[0],NewShape[1],3))
        TrainLabels=np.zeros(len(BatchSample)*Multiplier)
        i=0
        for Sample in BatchSample:
            #Images
            CenterImg=np.asarray(cv2.imread(Sample[0]))
            LeftImg=np.asarray(cv2.imread(Sample[1].strip()))  
            RightImg=np.asarray(cv2.imread(Sample[2].strip()))
            #PreProcess Images
            CenterImg=PreProcess(CenterImg)
            LeftImg=PreProcess(LeftImg)
            RightImg=PreProcess(RightImg)
            #Steering Angles
            CenterAngle=float(Sample[3])
            LeftAngle=(CenterAngle+SteerCorrection)
            RightAngle=(CenterAngle-SteerCorrection)
            #Image and Angle Accumulation
            TrainFeatures[i]=CenterImg
            TrainLabels[i]=CenterAngle
            i=i+1
            TrainFeatures[i]=LeftImg
            TrainLabels[i]=LeftAngle
            i=i+1
            TrainFeatures[i]=RightImg
            TrainLabels[i]=RightAngle
            yield sklearn.utils.shuffle(TrainFeatures,TrainLabels)

In [ ]:
#Converting into numpy Arrays
GeneratedTrain=BatchGenerator(TrainSamples,50)
GeneratedVal=BatchGenerator(ValidationSamples,50)


In [ ]:
#Network to drive the car
FilShape=[5,5,5,3,3]
NumFils=[24,36,48,64,64]
FCSize=[582,100,50,10]
model=Sequential()
#PreProcessing
model.add(Lambda(lambda x:x/255.0-0.5,input_shape=(NewShape[0],NewShape[1],3)))
#Crop 70 pixels on top and 25 pixels on bottom
model.add(Cropping2D(cropping=((30,12),(0,0))))
#CL1
model.add(Convolution2D(NumFils[0],FilShape[0],FilShape[0],activation='relu'
                        ,init='glorot_normal'))
#CL2
model.add(Convolution2D(NumFils[1],FilShape[1],FilShape[1],activation='relu'
                        ,init='glorot_normal'))
#CL3
model.add(Convolution2D(NumFils[2],FilShape[2],FilShape[2],activation='relu'
                        ,init='glorot_normal'))
#CL4
model.add(Convolution2D(NumFils[3],FilShape[3],FilShape[3],activation='relu'
                        ,init='glorot_normal'))
#CL5
model.add(Convolution2D(NumFils[4],FilShape[4],FilShape[4],activation='relu'
                        ,init='glorot_normal'))
#Flatten
model.add(Flatten())
#FC1
model.add(Dense(FCSize[0],activation='relu',init='glorot_normal'))
#model.add(Dropout(0.5))
#FC2
model.add(Dense(FCSize[1],activation='relu',init='glorot_normal'))
#model.add(Dropout(0.5))
#FC3
model.add(Dense(FCSize[2],activation='relu',init='glorot_normal'))
#model.add(Dropout(0.5))
#FC4
model.add(Dense(FCSize[3],activation='relu',init='glorot_normal'))
model.add(Dense(1))


In [ ]:
#Model
model.compile(loss='mse',optimizer='adam')
model.fit_generator(GeneratedTrain,samples_per_epoch=len(TrainSamples),validation_data=GeneratedVal,
                    nb_val_samples=len(ValidationSamples),nb_epoch=4)
print("Saving Model...")
model.save("Model.h5")
print("Done...")